In [1]:
import numpy as np

In [2]:
from tqdm import tqdm

In [3]:
import pyiron_contrib

In [4]:
from pyiron_base import state

In [5]:
from pyiron_atomistics import Project

In [6]:
def body_order(n=32, b=5):
    if b == 2:
        return [[i ,n-i] for i in range(n+1)]
    else:
        lst = []
        for i in range(n+1):
            for j in body_order(n=n-i, b=b-1):
                lst.append([i] + j)
        return lst

In [7]:
state.settings.configuration

{'user': 'pyiron',
 'resource_paths': ['/Users/jan/pyiron/resources',
  '/Users/jan/miniforge3/share/pyiron'],
 'project_paths': [],
 'connection_timeout': 60,
 'sql_connection_string': None,
 'sql_table_name': 'jobs_pyiron',
 'sql_view_connection_string': None,
 'sql_view_table_name': None,
 'sql_view_user': None,
 'sql_view_user_key': None,
 'sql_file': '/Users/jan/pyiron.db',
 'sql_host': None,
 'sql_type': 'SQLite',
 'sql_user_key': None,
 'sql_database': None,
 'project_check_enabled': False,
 'disable_database': True,
 'credentials_file': None,
 'write_work_dir_warnings': True,
 'config_file_permissions_warning': True}

In [8]:
project = Project('workflow')

In [9]:
project.remove_jobs(recursive=True, silently=True)

In [10]:
structure = project.create.structure.ase.bulk("Al", cubic=True).repeat([2,2,2])

In [11]:
element_lst = ["Fe", "Ni", "Cr", "Co", "Cu"]

In [12]:
number_atoms = len(structure)

In [13]:
number_species = len(element_lst)

In [14]:
cores = 1

In [15]:
potential = '2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1'

In [16]:
lst_32 = body_order(n=number_atoms, b=number_species)[:10]

In [17]:
structure_collect_lst, elastic_collect_lst = [], []

In [18]:
for conc_lst in tqdm(lst_32):
    # Generate structure 
    if sum(np.array(conc_lst)==0) != 4:
        mole_fraction = {
            el: conc/number_atoms
            for el, conc in zip(element_lst, conc_lst)
            if conc > 0
        }
        job = project.create_job(project.job_type.SQSJobWithoutOutput, "sqs")
        job._interactive_disable_log_file = True
        job.structure = structure
        job.input['mole_fractions'] = mole_fraction
        job.input['iterations'] = 1e6
        job.server.cores = cores
        job.run()
        structure_next = job._lst_of_struct[-1]
    else:
        structure_next = structure.copy()
        structure_next.symbols[:] = np.array(element_lst)[np.array(conc_lst)!=0][0] 
    structure_collect_lst.append(structure_next)
    
    # Minimize strucute 
    lmp_mini1 = project.create_job(project.job_type.LammpsInteractiveWithoutOutput, "lmp_mini", delete_existing_job=True)
    lmp_mini1.structure = structure_next
    lmp_mini1.potential = potential
    lmp_mini1.calc_minimize(pressure=0.0)
    lmp_mini1.server.run_mode.interactive = True
    lmp_mini1._interactive_disable_log_file = True  # disable lammps.log 
    keys = list(lmp_mini1.interactive_output_functions.keys())
    for k in keys:
        if k not in ['cells', 'positions', 'indices']:
            del lmp_mini1.interactive_output_functions[k]
    lmp_mini1.run()
    lmp_mini1.interactive_close()

    # Elastic constants 
    lmp_elastic = project.create_job(project.job_type.LammpsInteractiveWithoutOutput, "lmp_elastic", delete_existing_job=True)
    lmp_elastic.structure = lmp_mini1.get_structure()
    lmp_elastic.potential = potential
    lmp_elastic.interactive_enforce_structure_reset = True 
    lmp_elastic.server.run_mode.interactive = True
    lmp_elastic._interactive_disable_log_file = True  # disable lammps.log
    keys = list(lmp_elastic.interactive_output_functions.keys())
    for k in keys:
        if k not in ['energy_tot']:
            del lmp_elastic.interactive_output_functions[k]
    elastic = lmp_elastic.create_job(project.job_type.ElasticMatrixJobWithoutFiles, "elastic", delete_existing_job=True)
    elastic._interactive_disable_log_file = True  # disable lammps.log
    elastic.run()
    
    elastic_collect_lst.append(elastic._data["C"])

  0%|                                                    | 0/10 [00:00<?, ?it/s]

The job lmp_mini was saved and received the ID: lmp_mini


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


The job elastic was saved and received the ID: elastic
The job elastic_lmp_elastic was saved and received the ID: elastic_lmp_elastic


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 10%|████▍                                       | 1/10 [00:02<00:22,  2.53s/it]

...

/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 90%|███████████████████████████████████████▌    | 9/10 [00:38<00:04,  4.99s/it]

The job sqs was saved and received the ID: sqs
The job lmp_mini was saved and received the ID: lmp_mini


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


The job elastic was saved and received the ID: elastic
The job elastic_lmp_elastic was saved and received the ID: elastic_lmp_elastic


/Users/jan/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
100%|███████████████████████████████████████████| 10/10 [00:44<00:00,  4.43s/it]


In [19]:
elastic_collect_lst

[array([[171.55080699, 116.38765601, 116.38765601,   0.        ,
           0.        ,   0.        ],
        [116.38765601, 171.55080699, 116.38765601,   0.        ,
           0.        ,   0.        ],
        [116.38765601, 116.38765601, 171.55080699,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,  76.35007615,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
          76.35007615,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,  76.35007615]]),
 array([[168.20637662, 117.34259605, 117.34259605,   0.        ,
           0.        ,   0.        ],
        [117.34259605, 168.20637662, 117.34259605,   0.        ,
           0.        ,   0.        ],
        [117.34259605, 117.34259605, 168.20637662,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,  78.13518569,
      

In [20]:
[len(struct) for struct in structure_collect_lst]

[32, 32, 32, 32, 32, 32, 32, 32, 32, 32]